# <b style='color:magenta;'> Log Reg </b>
* Este Jupyter tem como função treinar uma `Regessão Logística` com os melhores parametros determinados por um `RandomSearch`.

-> Dataset completo com voluntários femininos e masculinos, sem outliers e sem a coluna de semanas e com pca.

---

Bibliotecas/Módulos

In [1]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle as pkl
import seaborn as sns
import sys

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, matthews_corrcoef, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

Configurações dos logs

In [2]:
logger = logging.getLogger("LogReg")
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('%(name)s - %(levelname)s - [+] ------- %(message)s -------') 
handler.setFormatter(formatter)
logger.handlers = [handler]

Cores

In [3]:
RED = "\033[1;31m"
BLUE = "\033[1;34m"
GREEN = "\033[1;32m"
PINK = "\033[1;45m"
HEADER = '\033[95m'
OKBLUE = '\033[94m'
OKCYAN = '\033[96m'
OKGREEN = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
ENDC = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
MAG = "\033[1;45m"

---

In [4]:
df = pd.read_csv("dados_FEM_10_voluntarios_c_semanas_10_sinais.csv")

In [5]:
df.sample()

,0,1,2,3,4,5,6,7,8,9,...,1992,1993,1994,1995,1996,1997,1998,1999,label,semana
2680,30,36,34,26,31,29,29,32,32,27,...,38,35,33,33,36,34,40,50,0,1


Verificando tamanho do dataset

In [6]:
len(df)

12800

* Seleção da variável target e das features

In [7]:
y = df["semana"]

In [8]:
df = df.drop(["semana"], axis=1)

#### PCA
Aplicando o PCA para reduzir a dimensão dos dados.

In [9]:
pca = PCA(n_components=1000)

In [10]:
pca.fit(df)

PCA(n_components=1000)

In [11]:
%%time

pca_samples = pca.transform(df)

CPU times: user 10 s, sys: 1.7 s, total: 11.7 s
Wall time: 911 ms


In [12]:
ps = pd.DataFrame(pca_samples)
ps.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-631.363457,28.230674,-3.659224,-97.899749,-72.264304,21.272170,68.910607,-6.372685,-35.955205,118.090223,...,4.447695,13.386609,3.949760,9.368692,22.431364,-10.622776,-8.300431,17.680793,-15.869544,-0.784775
1,-632.707114,66.152960,15.405938,-111.461697,-89.054916,31.325686,55.006025,2.045112,-32.756870,89.481034,...,13.915213,9.705655,12.205520,-7.671332,2.674601,-18.457498,-1.207821,1.433354,-14.821868,4.376299
2,-646.168714,93.290640,30.485999,-134.490441,-99.129664,22.432677,43.040378,5.703441,-30.196064,66.737921,...,15.614134,5.540331,4.778126,-6.082755,2.959655,-24.227547,-1.416548,2.057032,7.107458,12.264057
3,-643.352457,116.252996,34.411949,-143.144341,-106.239208,16.528166,20.641757,15.282286,-21.223789,41.918949,...,9.007912,-5.467687,0.409457,-6.812075,0.113423,-22.467674,0.181179,-5.813768,13.980082,20.124975
4,-617.238819,114.587003,49.331099,-142.535229,-110.318611,8.461582,3.495896,20.487859,-17.737526,14.678165,...,-3.174370,-11.312858,0.109611,-3.062717,-8.008435,-17.087508,5.705226,-5.852806,22.108944,9.059693


In [13]:
len(ps)

12800

In [14]:
len(ps.columns)

1000

In [15]:
X = ps

Separando os dados em dados de treino e teste

In [16]:
x_treino, x_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=7)

----

## Treinamento

### Regressão Logística

Parametros do modelo escolhidos para otimização

In [49]:
values = {'C':[1, 10, 100], 'solver':['newton-cg', 'liblinear', 'sag', 'lbfgs']}

In [50]:
clf = LogisticRegression()

In [51]:
def best_params(values: dict, clf: 'classificador', x_treino: np.array, x_teste: np.array, y_treino: np.array, y_teste: np.array) -> list:
    '''
        Instancia um classificador de busca e procura os melhores parâmetros para o modelo.
        
        Params
        ------
        :values: parametros a serem testados
        :clf: instancia do classificador desejado
        :x_treino: dados de treino
        :x_teste: dados de teste
        :y_treino: dados de treino -labels
        :y_teste: dados de treino - labels
        
        Return
        ------
        :best_params: lista com os melhores parametros
    '''
    
    logger.info('Instanciando SVM')
    random_clf = RandomizedSearchCV(clf, param_distributions=values, n_iter=200, verbose=10)
    
    logger.info('Treinando SVM')
    random_clf.fit(x_treino, y_treino)
    
    logger.info('Predict SVM')
    y_random_clf_rl = random_clf.predict(x_teste)
    
    logger.info('Parametros SVM')
    print(RED, random_clf.get_params())
    
    logger.info('Best Params SVM')
    best_params = random_clf.best_params_
    print(BLUE, best_params)
    
    return best_params

In [52]:
%%time

try:
    best = best_params(values, clf, x_treino, x_teste, y_treino, y_teste)

except Exception as error:
    logger.warning('Aconteceu algum problema...')
    logger.critical(error)

else:
    logger.info('Ok')
    
finally:
    logger.info('Busca finalizada')

SVM - INFO - [+] ------- Instanciando SVM -------
SVM - INFO - [+] ------- Treinando SVM -------
/home/saraselis/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 12 is smaller than n_iter=200. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START C=1, solver=newton-cg......................................
[CV 1/5; 1/12] END .......C=1, solver=newton-cg;, score=0.699 total time= 3.2min
[CV 2/5; 1/12] START C=1, solver=newton-cg......................................


/home/saraselis/.local/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/saraselis/.local/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


[CV 2/5; 1/12] END .......C=1, solver=newton-cg;, score=0.697 total time= 4.5min
[CV 3/5; 1/12] START C=1, solver=newton-cg......................................


/home/saraselis/.local/lib/python3.8/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/home/saraselis/.local/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/saraselis/.local/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


...

[CV 4/5; 12/12] END ........C=100, solver=lbfgs;, score=0.677 total time=  10.0s
[CV 5/5; 12/12] START C=100, solver=lbfgs.......................................


/home/saraselis/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5; 12/12] END ........C=100, solver=lbfgs;, score=0.690 total time=  10.4s


/home/saraselis/.local/lib/python3.8/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/home/saraselis/.local/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/saraselis/.local/lib/python3.8/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
SVM - INFO - [+] ------- Predict SVM -------
SVM - INFO - [+] ------- Parametros SVM -------
SVM - INFO - [+] ------- Best Params SVM -------
SVM - INFO - [+] ------- Ok -------
SVM - INFO - [+] ------- Busca finalizada -------


 {'cv': None, 'error_score': nan, 'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__l1_ratio': None, 'estimator__max_iter': 100, 'estimator__multi_class': 'auto', 'estimator__n_jobs': None, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'lbfgs', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(), 'n_iter': 200, 'n_jobs': None, 'param_distributions': {'C': [1, 10, 100], 'solver': ['newton-cg', 'liblinear', 'sag', 'lbfgs']}, 'pre_dispatch': '2*n_jobs', 'random_state': None, 'refit': True, 'return_train_score': False, 'scoring': None, 'verbose': 10}
 {'solver': 'newton-cg', 'C': 1}
CPU times: user 16h 35min 10s, sys: 2h 23min 32s, total: 18h 58min 43s
Wall time: 2h 24min 29s


Melhores parâmetros:

In [53]:
best

{'solver': 'newton-cg', 'C': 1}

{'solver': 'newton-cg', 'C': 1}

----

### Regressão Logística

In [19]:
def classificador(x_treino, y_treino, x_teste, y_teste):
    '''
        Treina um classificador com os melhores parametros possíveis.
        
        Params
        ------
        :x_treino: dados de treino
        :x_teste: dados de teste
        :y_treino: dados de treino -labels
        :y_teste: dados de treino - labels
    
        Return
        ------
        :LogReg: Modelo LogReg treinado
        :resultados (matrix): métricas
    '''
    
    logger.info('Instanciando LogReg')
    LogReg = OneVsRestClassifier(LogisticRegression(solver= 'lbfgs', C= 100))
    
    logger.info('Treinando LogReg')
    LogReg.fit(x_treino, y_treino)
    
    logger.info('Predict')
    categorias = LogReg.predict(x_teste)
    
    print('Métricas: \n \n')
    
    logger.info('Acurácia')
    
    acc = accuracy_score(y_teste, categorias)
    print(OKCYAN, "Acc \n" + str(acc) + "\n")
    
    logger.info("Mcc")
    mcc = matthews_corrcoef(y_teste, categorias)
    print(WARNING, "Mcc \n" + str(mcc) + "\n")
    
    # logger.info("F1")
    # f1 = f1_score(y_teste, categorias)
    # print(GREEN, "F1 \n" + str(f1) + "\n")
    
    logger.info('Class Report')
    resultados = classification_report(y_teste, categorias)
    print(RED, "Class Report \n" + resultados + "\n")
    
    
    return LogReg

In [22]:
%%time

try:
    LogReg = classificador(x_treino,
                        y_treino,
                        x_teste,
                        y_teste)

except Exception as error:
    logger.warning('Aconteceu algum problema...')
    logger.critical(error)

else:
    logger.info('Ok')
    
finally:
    logger.info('Treino finalizado')

LogReg - INFO - [+] ------- Instanciando LR -------
LogReg - INFO - [+] ------- Treinando LR -------
/home/saraselis/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/saraselis/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for al

Métricas: 
 

 Acc 
0.598828125

 Mcc 
0.4666066522234802

 Class Report 
              precision    recall  f1-score   support

           1       0.55      0.63      0.58       635
           2       0.55      0.60      0.57       621
           3       0.64      0.61      0.62       668
           4       0.69      0.56      0.62       636

    accuracy                           0.60      2560
   macro avg       0.61      0.60      0.60      2560
weighted avg       0.61      0.60      0.60      2560


CPU times: user 1min 1s, sys: 22.7 s, total: 1min 24s
Wall time: 5.33 s


---

#### Exportando modelo

In [23]:
pkl.dump(LogReg, open('modelos/REG_LOG_dados_FEM_10_voluntarios_s_semanas_15_sinais_noout.pkl', 'wb'))